In [1]:
# IMPORT LIBS
import pandas as pd
import numpy as np
import time
import os
from glob import glob
from datetime import datetime
from logzero import logger
from IPython.display import display, HTML
import matplotlib.pyplot as plt
from collections import defaultdict

import timescaledb_model as tsdb
db = tsdb.TimescaleStockMarketModel('bourse', 'ricou', 'localhost', 'monmdp')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

CSS = """
.output {
    flex-direction: row;
}
"""
HTML('<style>{}</style>'.format(CSS))

Logs of timescaledb_model go to /tmp/bourse.log


In [2]:
def parse_filename(filename):
    return datetime.strptime(filename[-30:-4], "%Y-%m-%d %H:%M:%S.%f")

def erase_c(df: pd.DataFrame):
    df['last'] = df['last'].astype(str)
    df['last'] = df['last'].str.replace('(c)', '')
    df['last'] = df['last'].str.replace('(s)', '')
    df['last'] = df['last'].str.replace(' ', '')
    df['last'] = df['last'].astype(float)
    return df

def custom_sort_key(file_path: str):
    return file_path[-30:-4]

In [3]:
def get_market_id(market_name: str) -> int:
    return int(db.raw_query(f"SELECT id FROM markets WHERE alias = '{market_name}'")[0][0])

def get_companies_id():
    return db.df_query('SELECT id, symbol FROM companies', chunksize=None)

In [4]:
def compute_companies(df, stock_name, market_id, market_name):
    companies_df = df[['name','symbol']].copy().drop_duplicates()
    companies_df.reset_index(drop=True, inplace=True)
    companies_df['mid'] = market_id

    companies_df['pea'] = market_name == "peapme"
    companies_df['boursorama'] = stock_name == "boursorama"
    companies_df.loc[:, ["symbol_nf", "isin", "reuters", "sector"]] = None
    companies_df.reset_index(names="id", inplace=True)

    # Write inside table
    print(f"{len(set(df.name)) = }")
    display(companies_df.tail(10))
    db.df_write(companies_df, "companies", commit=True, index=False)
    #display(companies_df)

def compute_stocks(df, compagnies_df):
    stock_df = df[['symbol', 'last', 'volume']].copy().reset_index(names='date')

    # stock_df = stock_df[stock_df['volume']] # MAX INT?
    stock_df = stock_df.merge(compagnies_df, left_on='symbol', right_on='symbol', how='left')
    stock_df.drop(columns=['symbol'], inplace=True)
    stock_df.rename(columns={'last': 'value', 'id': 'cid'}, inplace=True)

    # Write inside table
    display(stock_df.tail(10))
    db.df_write(stock_df, 'stocks', commit=True, index=False)
    #display(stock_df)

def compute_daystocks(df, compagnies_df):
    daystocks_df = df.copy()

    # Convert the index to datetime if it's not already in datetime format
    daystocks_df.reset_index(names="timestamp", inplace=True)

    # Group by 'name' column
    grouped = daystocks_df.groupby(['name', 'symbol', daystocks_df['timestamp'].dt.date])

    # Apply aggregations
    daystocks_df = grouped.agg({
        "timestamp": "first",
        'last': ['first', 'last', np.min, np.max],
        'volume': np.sum,
    })

    # Rename columns for better clarity
    daystocks_df.columns = ['date', 'open', 'close', 'low', 'high', 'volume'] # ADD 'date',
    daystocks_df["date"] = daystocks_df["date"].dt.date

    compagnies_df.merge(compagnies_df, left_on='symbol', right_on='symbol', how='left')
    compagnies_df.drop(columns=['symbol'], inplace=True)
    compagnies_df.rename(columns={'id': 'cid'}, inplace=True)

    # Write inside table
    display(daystocks_df.tail(10))
    db.df_write(daystocks_df, "daystocks", commit=True, index=False)
    #display(daystocks_df)

In [5]:
# CURRENT DATA
df = None
data = {}

def store_file(files, website, market_name, market_id):
    if website.lower() != "boursorama":
        return

    for filename in files:
        #if db.is_file_done(filename): # Useless for now
        #    continue

        # Add in final fidctionary
        data[parse_filename(filename)] = pd.read_pickle(filename)

    # CONCAT FINAL DATA
    df = pd.concat(data).reset_index(level=1, drop=True)

    # CLEAN DATA
    df.drop_duplicates(inplace=True)
    df.index = pd.to_datetime(df.index)
    erase_c(df)

    # df.sort_index(inplace=True)
    # df = df[df.symbol == "1rPAC"] # HARDCODE

    compute_companies(df, website, market_id, market_name)

    compagnies_df = get_companies_id()
    display(compagnies_df)
    compute_stocks(df, compagnies_df)
    compute_daystocks(df, compagnies_df)


    # Write the files as processed
    # db.df_write(files_df,"file_done",index=False)
    files_df = pd.DataFrame({'name': files})
    display(files_df)


In [6]:
# MAIN ARGS
data_path = "../docker/data"
batch_range = 1000
files_count = 0

# MAIN PROCESS
logger.debug(f"Work in progress...")
starting_time = time.time()
for stock in os.listdir(data_path):
    stock_path = f"{data_path}/{stock}"

    for year in os.listdir(stock_path):
        market_pattern = f"{data_path}/{stock}/{year}/*.bz2"

        available_files = glob(market_pattern)
        available_files = sorted(available_files, key=custom_sort_key)

        market_files = defaultdict(list)

        # Organize files into separate lists based on the market name
        for file_path in available_files:
            market_name = os.path.basename(file_path).split()[0]
            market_files[market_name].append(file_path)

        # Work on the data with batches
        for market_name, market_paths in market_files.items():
            logger.debug(f"Loading {market_name} ({year}) data for {stock} in batches of {batch_range}.")
            market_starting_time = time.time()

            market_id = get_market_id(market_name)

            for i in range(0, len(market_paths), batch_range):
                store_file(market_paths[i: i + batch_range], stock, market_name, market_id)

            files_count += len(market_paths)
            subprocessing_time = round(time.time() - market_starting_time, 3)
            logger.debug(f"Processing complete for {market_name} data in {year} with {len(market_paths)}"
                         f"files, completed in {subprocessing_time} seconds.")

rounded_processing_time = round(time.time() - starting_time, 3)
logger.debug(f"Work done on {files_count} files, in {rounded_processing_time} seconds.")

[D 240329 02:47:58 1468316835:7] Work in progress...
[D 240329 02:47:58 1468316835:27] Loading amsterdam (2023) data for boursorama in batches of 1000.


len(set(df.name)) = 4756


,id,name,symbol,mid,pea,boursorama,symbol_nf,isin,reuters,sector
4786,4786,1ST SEACOAS BNC,FSEA,6,False,True,None,None,None,None
4787,4787,APTORUM,APM,6,False,True,None,None,None,None
4788,4788,ATMV,ATMV,6,False,True,None,None,None,None
4789,4789,ATMV RTS 19.06.24,ATMVR,6,False,True,None,None,None,None
4790,4790,ALARUM SP ADR,SFET,6,False,True,None,None,None,None
4791,4791,BRIDGER,BAER,6,False,True,None,None,None,None
4792,4792,HSPO,HSPO,6,False,True,None,None,None,None
4793,4793,HSPO RTS 02.03.24,HSPOR,6,False,True,None,None,None,None
4794,4794,ALARUM SP ADR,ALAR,6,False,True,None,None,None,None
4795,4795,GENELUX,GNLX,6,False,True,None,None,None,None


IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "companies_pkey"
DETAIL:  Key (id)=(0) already exists.

[SQL: INSERT INTO companies (id, name, symbol, mid, pea, boursorama, symbol_nf, isin, reuters, sector) VALUES (%(id_m0)s, %(name_m0)s, %(symbol_m0)s, %(mid_m0)s, %(pea_m0)s, %(boursorama_m0)s, %(symbol_nf_m0)s, %(isin_m0)s, %(reuters_m0)s, %(sector_m0)s), (%(id_m1)s, %(name_m1)s, %(symbol_m1)s, %(mid_m1)s, %(pea_m1)s, %(boursorama_m1)s, %(symbol_nf_m1)s, %(isin_m1)s, %(reuters_m1)s, %(sector_m1)s), (%(id_m2)s, %(name_m2)s, %(symbol_m2)s, %(mid_m2)s, %(pea_m2)s, %(boursorama_m2)s, %(symbol_nf_m2)s, %(isin_m2)s, %(reuters_m2)s, %(sector_m2)s), (%(id_m3)s, %(name_m3)s, %(symbol_m3)s, %(mid_m3)s, %(pea_m3)s, %(boursorama_m3)s, %(symbol_nf_m3)s, %(isin_m3)s, %(reuters_m3)s, %(sector_m3)s), (%(id_m4)s, %(name_m4)s, %(symbol_m4)s, %(mid_m4)s, %(pea_m4)s, %(boursorama_m4)s, %(symbol_nf_m4)s, %(isin_m4)s, %(reuters_m4)s, %(sector_m4)s), (%(id_m

In [ ]:
"""show_data = ["low", "high"]
def plot_daystocks(dataframe):
    copy = dataframe.copy()
    copy.plot(y=show_data, figsize=(10,6), title=f"Cours de {companie_focus} pour {available_years}")
    plt.xlabel("Date")
    plt.ylabel("Cours en Euro (€)")
    plt.show()

plot_daystocks(result_daystocks)"""